### Integrantes 

- **Nome:** Lucas dos Santos 
    - Email: lsar@cin.ufpe.br

- **Nome:** Davi da Silva
    - Email: dsc6@cin.ufpe.br

In [242]:
import pandas as pd
from scipy.io import arff
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [243]:
# Carregar o arquivo .arff
data, meta = arff.loadarff('../data/speeddating.arff')

In [244]:
# Função para decodificar valores binários
def decode_value(value):
    if isinstance(value, bytes):
        return value.decode('utf-8')
    return value

In [245]:
# Converter para DataFrame do pandas
df = pd.DataFrame(data)

In [246]:
# Aplicar a função a todas as células do DataFrame
df = df.applymap(decode_value)

C:\Users\lucas\AppData\Local\Temp\ipykernel_8652\132754000.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(decode_value)


In [247]:
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1.0,female,21.0,27.0,6.0,[4-6],Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,[0-3],[3-5],7.0,6.0,[6-8],[5-6],0.0,1,0,0
1,0,1.0,female,21.0,22.0,1.0,[0-1],Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,[0-3],[3-5],7.0,5.0,[6-8],[5-6],1.0,1,0,0
2,1,1.0,female,21.0,22.0,1.0,[0-1],Asian/Pacific Islander/Asian-American,Asian/Pacific Islander/Asian-American,1,...,[0-3],[3-5],7.0,NaN,[6-8],[0-4],1.0,1,1,1
3,0,1.0,female,21.0,23.0,2.0,[2-3],Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,[0-3],[3-5],7.0,6.0,[6-8],[5-6],0.0,1,1,1
4,0,1.0,female,21.0,24.0,3.0,[2-3],Asian/Pacific Islander/Asian-American,Latino/Hispanic American,0,...,[0-3],[3-5],6.0,6.0,[6-8],[5-6],0.0,1,1,1


In [248]:
df.to_csv('../data/speeddating.csv', index=False, sep=';')

# Preparação dos dados

### 1. Tratamento de valores faltantes

1. *Verificar o Tipo de Dados*: Identificar as colunas com valores ausentes e verificar o tipo de dados para determinar o melhor método de preenchimento.

In [249]:
# Verificar colunas com valores ausentes e seus tipos de dados
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0]) # Colunas com valores ausentes
print(df.dtypes) # Tipos de dados das colunas

age                                95
age_o                             104
importance_same_race               79
importance_same_religion           79
pref_o_attractive                  89
pref_o_sincere                     89
pref_o_intelligence                89
pref_o_funny                       98
pref_o_ambitious                  107
pref_o_shared_interests           129
attractive_o                      212
sinsere_o                         287
intelligence_o                    306
funny_o                           360
ambitous_o                        722
shared_interests_o               1076
attractive_important               79
sincere_important                  79
intellicence_important             79
funny_important                    89
ambtition_important                99
shared_interests_important        121
attractive                        105
sincere                           105
intelligence                      105
funny                             105
ambition    

2. *Preenchimento das Variáveis Numéricas*: Preencher as variáveis numéricas com a média ou mediana. nosso caso vamos usar a mediana, pois ela é menos sensível a outliers.

- foi verificado que existe colunas com intervalos de valores, vamos tratar essas colunas

In [250]:
# Função para converter intervalo de string para valor médio
def convert_interval_to_mean(value):
    if isinstance(value, str) and '-' in value:
        try:
            # Remove colchetes e divide o intervalo
            parts = value.strip('[]').split('-')
            # Lida com valores negativos adequadamente ao identificar o primeiro elemento vazio
            if parts[0] == '':  # Caso o valor seja negativo (por exemplo, [-1-0])
                low, high = -float(parts[1]), float(parts[2])
            else:
                low, high = float(parts[0]), float(parts[1])
            # Calcula a média
            return (low + high) / 2
        except ValueError:
            return value  # Retorna o valor original se não puder ser convertido
    return value  # Retorna o valor original se não for intervalo

In [251]:
import re
# Função para verificar se uma coluna contém valores no formato de intervalo
def is_interval(value):
    return bool(re.match(r'\[\d+-\d+\]', str(value)))

# Lista para armazenar os nomes das colunas com valores de intervalo
interval_columns = []

# Iterar sobre as colunas do DataFrame
for column in df.columns:
    # Verificar se a coluna contém valores de intervalo
    if df[column].apply(is_interval).any():
        interval_columns.append(column)

interval_columns.append('d_interests_correlate')

In [252]:
# Aplicar a função nas colunas com intervalos
for col in interval_columns:
    df[col] = df[col].apply(convert_interval_to_mean)

In [253]:
df.select_dtypes(include='number').head()

,wave,age,age_o,d_age,d_d_age,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,pref_o_attractive,...,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met
0,1.0,21.0,27.0,6.0,5.0,2.0,4.0,3.5,3.5,35.0,...,2.0,4.0,2.0,1.5,4.0,7.0,6.0,7.0,5.5,0.0
1,1.0,21.0,22.0,1.0,0.5,2.0,4.0,3.5,3.5,60.0,...,2.0,4.0,2.0,1.5,4.0,7.0,5.0,7.0,5.5,1.0
2,1.0,21.0,22.0,1.0,0.5,2.0,4.0,3.5,3.5,19.0,...,2.0,4.0,2.0,1.5,4.0,7.0,NaN,7.0,2.0,1.0
3,1.0,21.0,23.0,2.0,2.5,2.0,4.0,3.5,3.5,30.0,...,2.0,4.0,2.0,1.5,4.0,7.0,6.0,7.0,5.5,0.0
4,1.0,21.0,24.0,3.0,2.5,2.0,4.0,3.5,3.5,30.0,...,2.0,4.0,2.0,1.5,4.0,6.0,6.0,7.0,5.5,0.0


- Visto que tem colunas com tipos de dados errados vamos corrigir esses tipos de dados

In [254]:
df['d_interests_correlate'] = df['d_interests_correlate'].astype(float)
df['decision'] = df['decision'].astype(int)
df['decision_o'] = df['decision_o'].astype(int)
df['match'] = df['match'].astype(int)
df['samerace'] = df['samerace'].astype(int)
df['has_null'] = df.isnull().any(axis=1).astype(int)

- Agora podemos prosseguir com o preenchimento dos valores faltantes na categoria de variáveis numéricas

In [255]:
# Preenchendo valores numéricos com a mediana
num_columns = df.select_dtypes(include=['float64', 'int64', 'number']).columns
for col in num_columns:
    df[col] = df[col].fillna(df[col].median()) # Preencher com a mediana

print(df[num_columns].isnull().sum()) # Verificar se ainda existem valores ausentes

has_null              0
wave                  0
age                   0
age_o                 0
d_age                 0
                     ..
d_guess_prob_liked    0
met                   0
decision              0
decision_o            0
match                 0
Length: 119, dtype: int64


3. *Preenchimento das Variáveis Categóricas*: Preencher as variáveis categóricas com o valor mais frequente, ou seja, a moda.

In [256]:
# Preenchendo valores categóricos com a moda
cat_columns = df.select_dtypes(include=['object']).columns
for col in cat_columns:
    df[col] = df[col].fillna(df[col].mode()[0])
    
print(df[cat_columns].isnull().sum()) # Verificar se ainda existem valores ausentes

gender    0
race      0
race_o    0
field     0
dtype: int64


In [257]:
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1.0,female,21.0,27.0,6.0,5.0,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,1.5,4.0,7.0,6.0,7.0,5.5,0.0,1,0,0
1,0,1.0,female,21.0,22.0,1.0,0.5,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,1.5,4.0,7.0,5.0,7.0,5.5,1.0,1,0,0
2,1,1.0,female,21.0,22.0,1.0,0.5,Asian/Pacific Islander/Asian-American,Asian/Pacific Islander/Asian-American,1,...,1.5,4.0,7.0,5.0,7.0,2.0,1.0,1,1,1
3,0,1.0,female,21.0,23.0,2.0,2.5,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,1.5,4.0,7.0,6.0,7.0,5.5,0.0,1,1,1
4,0,1.0,female,21.0,24.0,3.0,2.5,Asian/Pacific Islander/Asian-American,Latino/Hispanic American,0,...,1.5,4.0,6.0,6.0,7.0,5.5,0.0,1,1,1


- Foi identificado algumas colunas com valores do tipo intervalo com isso tratarmos essas colunas para que possamos preencher os valores faltantes, as colunas foram convertidas para o tipo de dados numéricos.

- o preenchimento dos valores faltantes foi feito com a moda das colunas para as variáveis categóricas e com a mediana para as variáveis numéricas.

___

### 2. Conversão de Tipos de Dados

- Ja anteriormente foi feito a conversão de algumas colunas para o tipo de dados numéricos, agora vamos converter as colunas  de object para category.

In [258]:
cat_columns

df[cat_columns] = df[cat_columns].astype('category')

-  As variáveis do tipo object foram convertidas para o tipo category para otimizar o uso de memória e facilitar a análise de variáveis categóricas. Essa decisão permite uma manipulação mais eficiente dos dados nas próximas etapas.

___

### 3. Tratamento de Outliers

- Vamos identificar e tratar os outliers das variáveis numéricas, para isso vamos usar o método IQR (Interquartile Range) para identificar e remover os outliers.

In [259]:
num_columns

Index(['has_null', 'wave', 'age', 'age_o', 'd_age', 'd_d_age', 'samerace',
       'importance_same_race', 'importance_same_religion',
       'd_importance_same_race',
       ...
       'd_expected_num_interested_in_me', 'd_expected_num_matches', 'like',
       'guess_prob_liked', 'd_like', 'd_guess_prob_liked', 'met', 'decision',
       'decision_o', 'match'],
      dtype='object', length=119)

In [260]:
# Função para identificar e tratar outliers
def treat_outliers(df, column):
    # Calcula o primeiro e terceiro quartil (Q1 e Q3)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define limites superior e inferior
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remoção de outliers (opcional)
    # df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    # Substituição de outliers pelo limite superior/inferior
    df[column] = df[column].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
    
    return df

In [261]:
# Aplicar o tratamento de outliers em todas as colunas numéricas
for col in num_columns:
    df = treat_outliers(df, col)

# Verificação do resultado
print(df.describe())

       has_null         wave          age        age_o        d_age  \
count    8378.0  8378.000000  8378.000000  8378.000000  8378.000000   
mean        1.0    11.350919    26.281213    26.286823     3.750895   
std         0.0     5.995903     3.294641     3.289388     2.931862   
min         1.0     1.000000    18.000000    18.000000     0.000000   
25%         1.0     7.000000    24.000000    24.000000     1.000000   
50%         1.0    11.000000    26.000000    26.000000     3.000000   
75%         1.0    15.000000    28.000000    28.000000     5.000000   
max         1.0    21.000000    34.000000    34.000000    11.000000   

           d_d_age     samerace  importance_same_race  \
count  8378.000000  8378.000000           8378.000000   
mean      4.237199     0.395799              3.777393   
std       3.790632     0.489051              2.833273   
min       0.500000     0.000000              0.000000   
25%       0.500000     0.000000              1.000000   
50%       2.500000

- Para tratar outliers em todas as colunas numéricas, aplicamos a regra do intervalo interquartil (IQR), que identifica valores extremos fora do intervalo Q1 − 1.5 ∗ IQR, Q3 + 1.5 ∗ IQR. Decidimos substituir esses valores pelo limite superior ou inferior do IQR, preservando assim o número de amostras na base. Essa abordagem assegura que valores extremos não distorçam a análise e a modelagem, mantendo a integridade do conjunto de dados.

In [262]:
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,1.0,1.0,female,21.0,27.0,6.0,5.0,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,1.5,4.0,7.0,6.0,7.0,5.5,0.0,1,0,0.0
1,1.0,1.0,female,21.0,22.0,1.0,0.5,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,1.5,4.0,7.0,5.0,7.0,5.5,0.0,1,0,0.0
2,1.0,1.0,female,21.0,22.0,1.0,0.5,Asian/Pacific Islander/Asian-American,Asian/Pacific Islander/Asian-American,1,...,1.5,4.0,7.0,5.0,7.0,2.0,0.0,1,1,0.0
3,1.0,1.0,female,21.0,23.0,2.0,2.5,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,1.5,4.0,7.0,6.0,7.0,5.5,0.0,1,1,0.0
4,1.0,1.0,female,21.0,24.0,3.0,2.5,Asian/Pacific Islander/Asian-American,Latino/Hispanic American,0,...,1.5,4.0,6.0,6.0,7.0,5.5,0.0,1,1,0.0


___

### 4. Criação de Novas Variáveis

- Vamos criar novas variáveis para melhorar a análise e a modelagem dos dados.

1. Diferenças Relacionais entre Atributos: Se há pares de variáveis relacionadas entre o participante e o par (ex. age e age_o), podemos criar novas variáveis que representam a diferença entre eles, como uma variável de diferença de idade.

In [263]:
# 1. Diferença de idade entre participante e par
df['age_difference'] = abs(df['age'] - df['age_o'])

C:\Users\lucas\AppData\Local\Temp\ipykernel_8652\1769004418.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['age_difference'] = abs(df['age'] - df['age_o'])


2. Transformações em Intervalos ou Binários: Variáveis contínuas podem ser transformadas em categorias para criar segmentos que facilitem a análise. Por exemplo, criar categorias para idade ou atratividade com base em intervalos.

In [264]:
# 2. Criação de grupos etários
df['age_group'] = pd.cut(df['age'], bins=[18, 25, 35, 50], labels=['18-25', '26-35', '36-50'])

C:\Users\lucas\AppData\Local\Temp\ipykernel_8652\1872149184.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['age_group'] = pd.cut(df['age'], bins=[18, 25, 35, 50], labels=['18-25', '26-35', '36-50'])


3. Interações entre Preferências e Avaliações: Combine variáveis de preferências para entender as interações entre características, como attractive e funny, e se há uma relação combinada que favoreça o "match".

In [265]:
# 3. Interação entre atratividade e diversão
df['attractive_funny_interaction'] = df['attractive'] * df['funny']

C:\Users\lucas\AppData\Local\Temp\ipykernel_8652\3918463293.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['attractive_funny_interaction'] = df['attractive'] * df['funny']


- Criação de Novas Variáveis

- Para enriquecer o conjunto de dados e explorar interações complexas que podem impactar a análise e modelagem foi feito a criação dessas novas variáveis:

1. Diferença de Idade: Criamos uma variável que mede a diferença de idade entre o participante e o par, o que pode indicar preferências relacionadas a idade.

2. Grupos Etários: A variável age foi segmentada em grupos etários para facilitar a análise de comportamentos entre diferentes faixas de idade.

3. Interação Atratividade e Diversão: Multiplicamos attractive e funny para criar uma variável de interação que explora se a combinação desses fatores aumenta as chances de um match.

Essas variáveis novas ajudam a capturar nuances e relações entre os dados que não seriam visíveis individualmente, permitindo insights mais profundos e aumentando o potencial para modelagens preditivas.

In [266]:
df[['age_difference', 'age_group', 'attractive_funny_interaction']]

,age_difference,age_group,attractive_funny_interaction
0,6.0,18-25,48.0
1,1.0,18-25,48.0
2,1.0,18-25,48.0
3,2.0,18-25,48.0
4,3.0,18-25,48.0
...,...,...,...
8373,1.0,18-25,56.0
8374,1.0,18-25,56.0
8375,4.0,18-25,56.0
8376,3.0,18-25,56.0


___

### 5. Normalização e Escalonamento de Variáveis

1. *Normalização (Escala de 0 a 1)*: Esta técnica transforma os valores para um intervalo entre 0 e 1, o que é útil quando queremos manter a distribuição relativa dos dados.
- Usamos a normalização em dados onde a interpretação em uma escala absoluta não é tão crítica, como attractive, funny, etc.

In [267]:
# Identificar colunas numéricas que precisam de normalização
columns_to_normalize = ['attractive', 'funny', 'intelligence', 'ambition', 'age_difference']

# Inicializar normalizador
normalizer = MinMaxScaler()

# Aplicar normalização (0 a 1)
df[columns_to_normalize] = normalizer.fit_transform(df[columns_to_normalize])

df[columns_to_normalize].head()

,attractive,funny,intelligence,ambition,age_difference
0,0.428571,0.428571,0.666667,0.5,0.428571
1,0.428571,0.428571,0.666667,0.5,0.071429
2,0.428571,0.428571,0.666667,0.5,0.071429
3,0.428571,0.428571,0.666667,0.5,0.142857
4,0.428571,0.428571,0.666667,0.5,0.214286


2. *Padronização (Média 0 e Desvio Padrão 1)*: A padronização transforma os dados para que tenham média zero e desvio padrão igual a um. É útil para variáveis que têm uma distribuição normal e para métodos que requerem dados centrados.

- A padronização é geralmente aplicada em variáveis com uma ampla faixa de valores ou quando queremos remover diferenças de escala entre variáveis.

In [268]:
columns_to_standardize = ['age', 'age_o', 'd_age', 'd_d_age']

standardizer = StandardScaler()

# Aplicar padronização (média 0 e desvio padrão 1)
df[columns_to_standardize] = standardizer.fit_transform(df[columns_to_standardize])

df[columns_to_standardize].head()

,age,age_o,d_age,d_d_age
0,-1.603066,0.216824,0.767171,0.201245
1,-1.603066,-1.303306,-0.938332,-0.985963
2,-1.603066,-1.303306,-0.938332,-0.985963
3,-1.603066,-0.999280,-0.597231,-0.458315
4,-1.603066,-0.695254,-0.256131,-0.458315


Para garantir que as variáveis numéricas estejam na mesma escala e melhorar o desempenho de modelos que são sensíveis a escalas, aplicamos duas técnicas principais:

1. Normalização: Transformamos as variáveis attractive, funny, intelligence, ambition e age_difference para um intervalo entre 0 e 1. Isso preserva a distribuição relativa dos valores e facilita a interpretação ao manter todas as variáveis dentro de um intervalo fixo.

2. Padronização: A variável age foi padronizada para ter média 0 e desvio padrão 1. Esse método é particularmente útil para variáveis que têm ampla faixa de valores e para assegurar que diferenças de escala não influenciem os resultados.

Essas transformações ajudam a evitar que variáveis com amplitudes maiores dominem o modelo e facilitam o uso de algoritmos baseados em distância, garantindo uma análise consistente e equilibrada entre as variáveis.

In [269]:
df.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match,age_difference,age_group,attractive_funny_interaction
0,1.0,1.0,female,-1.603066,0.216824,0.767171,0.201245,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,6.0,7.0,5.5,0.0,1,0,0.0,0.428571,18-25,48.0
1,1.0,1.0,female,-1.603066,-1.303306,-0.938332,-0.985963,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,5.0,7.0,5.5,0.0,1,0,0.0,0.071429,18-25,48.0
2,1.0,1.0,female,-1.603066,-1.303306,-0.938332,-0.985963,Asian/Pacific Islander/Asian-American,Asian/Pacific Islander/Asian-American,1,...,5.0,7.0,2.0,0.0,1,1,0.0,0.071429,18-25,48.0
3,1.0,1.0,female,-1.603066,-0.999280,-0.597231,-0.458315,Asian/Pacific Islander/Asian-American,European/Caucasian-American,0,...,6.0,7.0,5.5,0.0,1,1,0.0,0.142857,18-25,48.0
4,1.0,1.0,female,-1.603066,-0.695254,-0.256131,-0.458315,Asian/Pacific Islander/Asian-American,Latino/Hispanic American,0,...,6.0,7.0,5.5,0.0,1,1,0.0,0.214286,18-25,48.0


___

### 6. Codificação de Variáveis Categóricas

- o objetivo é transformar as variáveis categóricas em um formato numérico que possa ser utilizado em modelos de machine learning, já que a maioria dos algoritmos não aceita variáveis do tipo object ou category.

1 One-Hot Encoding:

- Transforme cada categoria em uma coluna binária (0 ou 1). Isso é útil para variáveis categóricas com poucas categorias e evita a introdução de ordens onde não há.

- Exemplo: A variável gender com categorias "female" e "male" será transformada em duas colunas (gender_female e gender_male).

In [270]:
cat_columns

Index(['gender', 'race', 'race_o', 'field'], dtype='object')

In [271]:
# Aplicar One-Hot Encoding
df = pd.get_dummies(df, columns=cat_columns, drop_first=True)
df.head()

,has_null,wave,age,age_o,d_age,d_d_age,samerace,importance_same_race,importance_same_religion,d_importance_same_race,...,field_physics [astrophysics],field_political science,field_psychology,field_psychology and english,field_social work,field_sociology,field_speech pathology,field_teaching of English,field_theory,field_working
0,1.0,1.0,-1.603066,0.216824,0.767171,0.201245,0,2.0,4.0,3.5,...,False,False,False,False,False,False,False,False,False,False
1,1.0,1.0,-1.603066,-1.303306,-0.938332,-0.985963,0,2.0,4.0,3.5,...,False,False,False,False,False,False,False,False,False,False
2,1.0,1.0,-1.603066,-1.303306,-0.938332,-0.985963,1,2.0,4.0,3.5,...,False,False,False,False,False,False,False,False,False,False
3,1.0,1.0,-1.603066,-0.999280,-0.597231,-0.458315,0,2.0,4.0,3.5,...,False,False,False,False,False,False,False,False,False,False
4,1.0,1.0,-1.603066,-0.695254,-0.256131,-0.458315,0,2.0,4.0,3.5,...,False,False,False,False,False,False,False,False,False,False


- Para preparar as variáveis categóricas para modelagem, aplicamos One-Hot Encoding nas variáveis gender, race e field, transformando cada categoria em uma coluna binária. A técnica de One-Hot Encoding foi escolhida para evitar a introdução de uma ordem artificial entre categorias e preservar a relação original dos dados.

- Ao utilizar drop_first=True, eliminamos uma categoria de cada variável para evitar multicolinearidade, mantendo a mesma informação. Esse processo é fundamental para garantir que todas as variáveis categóricas estejam em um formato numérico adequado para algoritmos de machine learning.

Com essa codificação, a base de dados está pronta para a modelagem, com todas as variáveis em formatos compatíveis para análise quantitativa e predição.

In [272]:
df.to_csv('../data/speeddating_preprocessed.csv', index=False, sep=';')